<a href="https://colab.research.google.com/github/joseorlandomx/topicodeindustria1/blob/main/Practica_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Práctica 4**

José Orlando Salas Contreras (208743738)

[jose.salas4373@alumnos.udg.mx](mailto:jose.salas4373@alumnos.udg.mx)

**Requisitos mínimos:**

* **Funciones** que generen trayectorias tipo Brownian Motion (**BM**), Correlated Random Walk (CRW) y Lévy Flight (LF).
* Cada una de las funciones deberá tomar como parámetros el **numero de pasos**, la **velocidad** y **posición inicial**.
  * Además la funciones para **CRW** y **LF** deberán tomar como parámetro el **coeficiente** para la distribución Cauchy.
  * Por último, la función para **LF** también deberá también aceptar como parámetro el **exponente Lévy** (alpha).
* **Funciones** para calcular las métricas **Path length** and **Mean Squared Displacement**.

**El dashboard degerá contener al menos los siguientes elementos:**

* Un panel para desplegar la trayectoria en 3D
* Un panel para desplegar la métrica de elección de la trayectoria.
* Drop down menu (u otro tipo de selector) que permita elegir la métrica a calcular de la trayectoria bajo análisis (path length o mean squared displacement).
* Radio Buttons (u otro tipo de selector) que permita elegir el tipo de trayectoria a generar y análizar.
* Widgets que permitan introducir los valores enteros o de punto flotante (segun sea el caso) para ajustar los parámetros de las trayectorias. Estos widgets deberán mostrarse de manera dinámica, es decir, solo deberan mostrarse los parametros relevantes para cada tipo de trayectoria.